In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

digit_recognizer_path = kagglehub.competition_download('digit-recognizer')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [ ]:
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split

In [ ]:
import torch.nn as nn

class DigitClassifier(nn.Module):
    def __init__(self):
        super(DigitClassifier, self).__init__()
        self.convnet = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=5, stride=1, padding=0),  # 32x32x1 -> 28x28x4
            #nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2),                # 28x28x4 -> 14x14x4

            nn.Conv2d(4, 16, kernel_size=5, stride=1, padding=0), # 14x14x4 -> 10x10x16
            #nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2)                 # 10x10x16 -> 5x5x16
        )
        self.fc1 = nn.Linear(400, 120)  # Fully connected layer (400 -> 120)
        self.fc2 = nn.Linear(120, 10)  # Fully connected layer (120 -> 10)
        self.activation = nn.Tanh()    # Activation for hidden layer

    def forward(self, x_in):
        """
        Args:
            x_in: input_image (32x32)
        """
        x = self.convnet(x_in)       # Pass through convolutional layers
        x = x.view(x.size(0), -1)    # Flatten for fully connected layers
        x = self.activation(self.fc1(x))  # Hidden layer with activation
        prediction = self.fc2(x)    # Output layer
        return prediction


In [ ]:
# Custom Dataset for train.csv
class DigitDataset(Dataset):
    def __init__(self, train_data):
        """
        Args:
            csv_file (str): Path to the train.csv file
        """
        self.data = train_data
        self.labels = self.data['label'].values  # Extract the labels
        self.images = self.data.drop(columns=['label']).values / 255.0  # Normalize pixel values to [0, 1]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the data point to retrieve
        Returns:
            Tuple of (image_tensor, label_tensor)
        """
        label = self.labels[idx]
        # Reshape the flat 784 pixels into 28x28 and add a channel dimension (1x28x28)
        image = self.images[idx].reshape(1, 28, 28).astype('float32')
        image = torch.tensor(image).unsqueeze(0)  # Add a batch dimension temporarily
        image = F.interpolate(image, size=(32, 32), mode='bilinear', align_corners=False)
        image = image.squeeze(0)  # Remove the batch dimension

        return image, torch.tensor(label, dtype=torch.long)

# Hyperparameters
batch_size = 64
learning_rate = 0.01
num_epochs = 10

# Load training data
full_dataset = DigitDataset(train_data)

# Compute lengths for train and validation sets
total_size = len(full_dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size

# Split the dataset
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Define model, loss function, and optimizer
model = DigitClassifier()  # Use the model defined earlier
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Training Phase
    model.train()
    total_loss = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {total_loss / len(train_loader):.4f}')

    # Validation Phase
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)  # Calculate validation loss
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss / len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%')

Epoch [1/10], Training Loss: 0.3725
Epoch [1/10], Validation Loss: 0.3459, Validation Accuracy: 89.17%
Epoch [2/10], Training Loss: 0.3881
Epoch [2/10], Validation Loss: 0.4462, Validation Accuracy: 86.30%
Epoch [3/10], Training Loss: 0.4019
Epoch [3/10], Validation Loss: 0.4272, Validation Accuracy: 86.95%
Epoch [4/10], Training Loss: 0.4195
Epoch [4/10], Validation Loss: 0.3698, Validation Accuracy: 88.48%
Epoch [5/10], Training Loss: 0.4044
Epoch [5/10], Validation Loss: 0.3428, Validation Accuracy: 89.35%
Epoch [6/10], Training Loss: 0.4006
Epoch [6/10], Validation Loss: 0.4003, Validation Accuracy: 88.33%
Epoch [7/10], Training Loss: 0.3892
Epoch [7/10], Validation Loss: 0.4220, Validation Accuracy: 87.38%
Epoch [8/10], Training Loss: 0.3910
Epoch [8/10], Validation Loss: 0.3981, Validation Accuracy: 87.76%
Epoch [9/10], Training Loss: 0.3830
Epoch [9/10], Validation Loss: 0.4120, Validation Accuracy: 87.27%
Epoch [10/10], Training Loss: 0.3674
Epoch [10/10], Validation Loss: 0.32

In [ ]:
# Custom Test Dataset (assuming test.csv has 784 columns of pixel values)
class TestDataset(Dataset):
    def __init__(self, test_data):
        """
        Args:
            csv_file (str): Path to the test.csv file
        """
        self.data = test_data.values / 255.0  # Normalize pixel values to [0, 1]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the data point to retrieve
        Returns:
            image_tensor (1x32x32)
        """
        image = self.data[idx].reshape(1, 28, 28).astype('float32')  # Reshape to 1x28x28
        image = torch.tensor(image).unsqueeze(0)  # Add a batch dimension temporarily
        image = F.interpolate(image, size=(32, 32), mode='bilinear', align_corners=False)
        image = image.squeeze(0)  # Remove the batch dimension
        return image

# Load the test data

test_dataset = TestDataset(test_data)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Make predictions
model.eval()  # Set the model to evaluation mode
predictions = []

with torch.no_grad():
    for images in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get predicted class
        predictions.extend(predicted.cpu().numpy())  # Collect predictions

# Create submission DataFrame
submission = pd.DataFrame({
    'ImageId': range(1, len(predictions) + 1),  # ImageId starts from 1
    'Label': predictions
})

# Save to CSV
submission.to_csv('submission.csv', index=False)
print("submission.csv generated!")

submission.csv generated!
